In [1]:
from sklearn.datasets import load_boston

from matplotlib import pyplot as plt

import numpy as np
from scipy.linalg import sqrtm
from scipy.linalg import svd

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.utils.extmath import randomized_svd, svd_flip

from skcosmo.linear_model import IncrementalRidge
from skcosmo.preprocessing import StandardFlexibleScaler
from skcosmo.utils.incremental import update_inverse_covariance

In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
The savefig.jpeg_quality rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
Th

In [2]:
#X, Y = load_boston(return_X_y=True)
X = np.random.rand(1000, 500)
Y = np.random.rand(1000)
Y = Y.reshape(-1, 1)

X_scaler = StandardFlexibleScaler(with_mean=True, with_std=True, column_wise=False)
X = X_scaler.fit_transform(X)

Y_scaler = StandardFlexibleScaler(with_mean=True, with_std=True, column_wise=True)
Y = Y_scaler.fit_transform(Y)

In [7]:
batch_size = 10
n_batches = len(X) // batch_size
if len(X) % n_batches > 0:
    n_batches += 1

In [8]:
alpha = 1.0E-12
mixing = 0.5
tol = 1.0E-12

#### Increment 1

In [5]:
%%timeit

C = np.zeros((X.shape[1], X.shape[1]))
XY = np.zeros((X.shape[1], Y.shape[1]))

ridge = Ridge(alpha=alpha)
ridge.fit(X, Y)
Yhat = ridge.predict(X)

for n in range(0, n_batches):
    batch_slice = slice(n*batch_size, (n+1)*batch_size)
    X_batch = X[batch_slice]
    Y_batch = Yhat[batch_slice]
    
    C += X_batch.T @ X_batch
    XY += X_batch.T @ Y_batch
    
    vC, UC = np.linalg.eigh(C)
    UC = UC[:, vC > tol]
    vC = vC[vC > tol]
    C_inv_sqrt = np.linalg.multi_dot([UC, np.diagflat(1.0 / np.sqrt(vC)), UC.T])
    
print(mixing * C + (1.0 - mixing) * np.linalg.multi_dot([C_inv_sqrt, XY, XY.T, C_inv_sqrt]))

[[ 1.00732307 -0.0646987  -0.09711756 ... -0.06655068 -0.2805608
  -0.02520541]
 [-0.0646987   1.1664797   0.20060443 ...  0.00654007  0.7717405
  -0.07380589]
 [-0.09711756  0.20060443  1.35386544 ...  0.04134889  1.02668046
  -0.07266016]
 ...
 [-0.06655068  0.00654007  0.04134889 ...  0.99748513  0.10649393
  -0.0409446 ]
 [-0.2805608   0.7717405   1.02668046 ...  0.10649393  4.346531
  -0.20212845]
 [-0.02520541 -0.07380589 -0.07266016 ... -0.0409446  -0.20212845
   1.01685193]]
[[ 1.00732307 -0.0646987  -0.09711756 ... -0.06655068 -0.2805608
  -0.02520541]
 [-0.0646987   1.1664797   0.20060443 ...  0.00654007  0.7717405
  -0.07380589]
 [-0.09711756  0.20060443  1.35386544 ...  0.04134889  1.02668046
  -0.07266016]
 ...
 [-0.06655068  0.00654007  0.04134889 ...  0.99748513  0.10649393
  -0.0409446 ]
 [-0.2805608   0.7717405   1.02668046 ...  0.10649393  4.346531
  -0.20212845]
 [-0.02520541 -0.07380589 -0.07266016 ... -0.0409446  -0.20212845
   1.01685193]]
[[ 1.00732307 -0.0646987

#### Increment 2

In [6]:
%%timeit

C_inv = alpha * np.eye(X.shape[1])
C = np.zeros((X.shape[1], X.shape[1]))
XY = np.zeros((X.shape[1], Y.shape[1]))

for n in range(0, n_batches):
    batch_slice = slice(n*batch_size, (n+1)*batch_size)
    X_batch = X[batch_slice]
    Y_batch = Y[batch_slice]
    
    C += X_batch.T @ X_batch
    XY += X_batch.T @ Y_batch
    C_sqrt = sqrtm(C)
    
    #_, vC, UC = np.linalg.eigh(C, full_matrices=False)
    #UC = UC[:, vC > tol]
    #vC = vC[vC > tol]
    #C_sqrt = np.linalg.multi_dot([UC, np.diagflat(np.sqrt(vC)), UC.T])
    
    C_inv = update_inverse_covariance(C_inv, X_batch)
    #C_inv = np.linalg.lstsq(C, np.eye(len(C)), rcond=None)[0]
    
print(mixing * C + (1.0 - mixing) * np.linalg.multi_dot([C_sqrt, C_inv, XY, XY.T, C_inv, C_sqrt]))    

[[ 0.96985334  0.01498868  0.01466167 ... -0.05412871  0.07528184
  -0.046044  ]
 [ 0.01498868  0.99700747 -0.03711796 ... -0.01987789  0.01496514
  -0.02948818]
 [ 0.01466167 -0.03711796  1.02040706 ...  0.00429184 -0.03486485
  -0.01049475]
 ...
 [-0.05412871 -0.01987789  0.00429184 ...  0.993367   -0.01147504
  -0.03403619]
 [ 0.07528184  0.01496514 -0.03486485 ... -0.01147504  0.96716342
  -0.00422848]
 [-0.046044   -0.02948818 -0.01049475 ... -0.03403619 -0.00422848
   1.00526266]]


KeyboardInterrupt: 